## Importing libraries

In [10]:
import numpy as np
from skorch import NeuralNetClassifier
from torch import cuda, nn
from torch.optim import Adam
from torchvision import models
from torchvision.transforms import Compose, ToTensor

from dlordinal.datasets import FGNet
from dlordinal.losses import TriangularLoss

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the `DataLoader`, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [11]:
optimiser_params = {"lr": 1e-3, "bs": 400, "epochs": 5, "s": 2, "c": 0.2, "beta": 0.5}

workers = 3

Now we use the `FGNet` method to download and preprocess the images.

In [12]:
fgnet_train = FGNet(
    root="./datasets",
    download=True,
    train=True,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)

fgnet_test = FGNet(
    root="./datasets",
    download=True,
    train=False,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)

num_classes = len(fgnet_train.classes)
classes = fgnet_train.classes
targets = fgnet_train.targets

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cpu device


## Estimator

We are setting up a deep learning model using `PyTorch` and `Skorch`. First, we define the model architecture using ResNet18, a pre-trained convolutional neural network, and customize its fully connected layer to match the number of classes in our classification task. Then we specify the loss function, in this case, a custom Triangular Cross Entropy Loss[1]. Finally, we configure the Skorch estimator, which serves as a bridge between PyTorch and scikit-learn, allowing us to train and evaluate our model seamlessly. We provide the model, loss function, and optimiser details such as the learning rate and number of epochs to the estimator. Additionally, we specify parameters for data loading and processing, like batch size and the number of workers, to optimise training performance.

[1]: Víctor Manuel Vargas, Pedro Antonio Gutiérrez, Javier Barbero-Gómez, and César Hervás-Martínez (2023). *Soft Labelling Based on Triangular Distributions for Ordinal Classification.* Information Fusion, 93, 258--267. doi.org/10.1016/j.inffus.2023.01.003

In [13]:
# Model
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss function
loss_fn = TriangularLoss(base_loss=nn.CrossEntropyLoss(), num_classes=num_classes).to(
    device
)

# Skorch estimator
estimator = NeuralNetClassifier(
    module=model,
    criterion=loss_fn,
    optimizer=Adam,
    lr=optimiser_params["lr"],
    max_epochs=optimiser_params["epochs"],
    train_split=None,
    callbacks=[],
    device=device,
    verbose=0,
    iterator_train__batch_size=optimiser_params["bs"],
    iterator_train__shuffle=True,
    iterator_train__num_workers=workers - 1,
    iterator_train__pin_memory=True,
    iterator_valid__batch_size=optimiser_params["bs"],
    iterator_valid__shuffle=False,
    iterator_valid__num_workers=workers - 1,
    iterator_valid__pin_memory=True,
)

In [14]:
estimator.fit(X=fgnet_train, y=targets)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(6

In [15]:
train_probs = estimator.predict_proba(fgnet_train)
print(f"Train probabilities = {train_probs=}\n")

test_probs = estimator.predict_proba(fgnet_test)
print(f"Test probabilities = {test_probs=}")

Train probabilities = train_probs=array([[-12.222035  ,  -9.642973  ,   3.7686963 ,   6.167635  ,
         -0.01739216,  -2.153393  ],
       [  6.183529  ,   0.04459053,  -1.9473094 ,  -1.4720759 ,
         -5.5622606 ,  -4.28996   ],
       [  0.66979164,  -3.3910298 ,  -1.2518983 ,   0.7819392 ,
         -1.7301369 ,  -2.261899  ],
       ...,
       [ -7.9363914 ,  -7.368971  ,   0.07903418,   2.3583612 ,
          2.210815  ,   0.3293348 ],
       [ -9.654899  ,  -9.874651  ,  -0.9765438 ,   2.999129  ,
          2.1711988 ,   2.5746014 ],
       [ -6.3013945 ,  -4.3200827 ,   1.3173927 ,   3.8813162 ,
          0.16160546,  -4.795178  ]], dtype=float32)

Test probabilities = test_probs=array([[ -7.986564 ,  -5.1126056,   2.5549955,   5.1918745,  -1.0416505,
         -3.2590356],
       [  4.134048 ,   0.306721 ,  -1.0112333,  -0.6821852,  -5.75443  ,
         -3.2740388],
       [ -2.4858363,  -0.8356784,   1.4894345,   5.404357 ,  -4.685725 ,
         -5.947282 ],
       ...,
  